In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

In [2]:
lemmatizer = WordNetLemmatizer()

with open('mentalhealth.json', 'r') as file:
    intents = json.load(file)

In [3]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.h5')

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [5]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [6]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key= lambda x:x[1],reverse=True)
    return_list = [] 
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
    return return_list

In [7]:
def get_response(intents_list,intents_json):
    if not intents_list:  
        return "I'm sorry, I didn't understand that. Could you please rephrase?"
    
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = i['responses']
            break
    return result

In [8]:
print("GO! ChatBot is runnning!")

GO! ChatBot is runnning!


In [ ]:
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints,intents)
    print(res)